In [ ]:
# !pip install datasets==3.6.0
from huggingface_hub import login
login("")

In [ ]:
from datasets import load_dataset, Dataset, Audio, concatenate_datasets, DatasetDict

In [ ]:
ds = load_dataset("Elormiden/RIK-cypriot-news-only", streaming=True)

In [ ]:
from tqdm import tqdm
import random

def sample_and_chunk_dataset(ds: DatasetDict, chunk_duration: float = 7.0, seed: int = 42) -> DatasetDict:
    random.seed(seed)
    total_rows = 125  # Original dataset size
    indices = list(range(total_rows))

    def chunk_generator():
        current_idx = 0
        for i, example in enumerate(tqdm(ds['train'], desc="Processing audio files")):
            if current_idx < len(indices) and i == indices[current_idx]:
                audio_array = example['audio']['array']
                sampling_rate = example['audio']['sampling_rate']
                chunk_samples = int(chunk_duration * sampling_rate)
                for j in range(0, len(audio_array), chunk_samples):
                    chunk = audio_array[j:j + chunk_samples]
                    if len(chunk) == chunk_samples:
                        yield {'audio': {'array': chunk, 'sampling_rate': sampling_rate}}
                current_idx += 1

    # Create and return new DatasetDict
    return DatasetDict({'train': Dataset.from_generator(chunk_generator)})

In [ ]:
new_ds = sample_and_chunk_dataset(ds)

In [ ]:
new_ds = new_ds.cast_column('audio', Audio(sampling_rate=16000))

In [ ]:
new_ds.push_to_hub("Elormiden/only-news-shuffled-cut")